In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import math
import os
from keras.layers import Input, Dense, Conv2D, Flatten, GaussianDropout, MaxPooling2D, Dropout 
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.losses import binary_crossentropy, categorical_crossentropy
from sklearn.metrics import cohen_kappa_score, f1_score, roc_auc_score, precision_score, recall_score, confusion_matrix
from keras.callbacks import Callback
from keras.applications.xception import Xception
from keras.layers import Input, Dense, Conv2D, Flatten, GaussianDropout, MaxPooling2D, Dropout, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam
from sklearn.utils import class_weight, shuffle
from keras.callbacks import Callback
from sklearn.model_selection import train_test_split
from keras.utils import Sequence
from keras.utils import to_categorical
import cv2

In [ ]:
print(os.listdir("../input/diabetic-retinopathy-resized/resized_train_cropped/resized_train_cropped/"))

In [ ]:
#df_train = pd.read_csv('../input/aptos2019-blindness-detection/train.csv')
df_train = pd.read_csv('../input/diabetic-retinopathy-resized/trainLabels_cropped.csv')

x = df_train['image'].tolist()
y = df_train['level'].tolist()

x, y = shuffle(x, y, random_state=8)
y = to_categorical(y, num_classes=5)

In [ ]:
train_x, valid_x, train_y, valid_y = train_test_split(x, y, test_size=0.15,
                                                      stratify=y, random_state=8)

In [ ]:
BATCH_SIZE = 32
SIZE = 299

In [ ]:
def cohen_ks(true, pred):
    return cohen_kappa_score(true, pred, labels=[0,1,2,3,4], weights='quadratic')

In [ ]:
# reference link: https://www.kaggle.com/christofhenkel/weighted-kappa-loss-for-keras-tensorflow
def kappa_loss(y_true, y_pred, y_pow=2, eps=1e-12, N=5, bsize=32, name='kappa'):
    """A continuous differentiable approximation of discrete kappa loss.
        Args:
            y_pred: 2D tensor or array, [batch_size, num_classes]
            y_true: 2D tensor or array,[batch_size, num_classes]
            y_pow: int,  e.g. y_pow=2
            N: typically num_classes of the model
            bsize: batch_size of the training or validation ops
            eps: a float, prevents divide by zero
            name: Optional scope/name for op_scope.
        Returns:
            A tensor with the kappa loss."""

    with tf.name_scope(name):
        y_true = tf.to_float(y_true)
        repeat_op = tf.to_float(tf.tile(tf.reshape(tf.range(0, N), [N, 1]), [1, N]))
        repeat_op_sq = tf.square((repeat_op - tf.transpose(repeat_op)))
        weights = repeat_op_sq / tf.to_float((N - 1) ** 2)
    
        pred_ = y_pred ** y_pow
        try:
            pred_norm = pred_ / (eps + tf.reshape(tf.reduce_sum(pred_, 1), [-1, 1]))
        except Exception:
            pred_norm = pred_ / (eps + tf.reshape(tf.reduce_sum(pred_, 1), [bsize, 1]))
    
        hist_rater_a = tf.reduce_sum(pred_norm, 0)
        hist_rater_b = tf.reduce_sum(y_true, 0)
    
        conf_mat = tf.matmul(tf.transpose(pred_norm), y_true)
    
        nom = tf.reduce_sum(weights * conf_mat)
        denom = tf.reduce_sum(weights * tf.matmul(
            tf.reshape(hist_rater_a, [N, 1]), tf.reshape(hist_rater_b, [1, N])) /
                              tf.to_float(bsize))
    
        return nom*0.5 / (denom + eps) + categorical_crossentropy(y_true, y_pred)*0.5

In [ ]:
class My_Generator(Sequence):

    def __init__(self, image_filenames, labels,
                 batch_size, is_train=False,
                 mix=False, augment=False):
        self.image_filenames, self.labels = image_filenames, labels
        self.batch_size = batch_size
        self.is_train = is_train
        self.is_augment = augment
        if(self.is_train):
            self.on_epoch_end()
        self.is_mix = mix

    def __len__(self):
        return int(np.ceil(len(self.image_filenames) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.image_filenames[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]

        if(self.is_train):
            return self.train_generate(batch_x, batch_y)
        return self.valid_generate(batch_x, batch_y)

    def on_epoch_end(self):
        if(self.is_train):
            self.image_filenames, self.labels = shuffle(self.image_filenames, self.labels)

    def train_generate(self, batch_x, batch_y):
        batch_images = []
        for (sample, label) in zip(batch_x, batch_y):
            #print(sample)
            img = cv2.imread('../input/diabetic-retinopathy-resized/resized_train_cropped/resized_train_cropped/'+sample+'.jpeg')
            img = cv2.resize(img, (SIZE, SIZE))
            batch_images.append(img)
        batch_images = np.array(batch_images, np.float32) / 255.
        batch_y = np.array(batch_y, np.float32)
        return batch_images, batch_y

    def valid_generate(self, batch_x, batch_y):
        batch_images = []
        for (sample, label) in zip(batch_x, batch_y):
            img = cv2.imread('../input/diabetic-retinopathy-resized/resized_train_cropped/resized_train_cropped/'+sample+'.jpeg')
            img = cv2.resize(img, (SIZE, SIZE))
            batch_images.append(img)
        batch_images = np.array(batch_images, np.float32)  / 255.
        batch_y = np.array(batch_y, np.float32)
        return batch_images, batch_y

In [ ]:
train_generator = My_Generator(train_x, train_y, BATCH_SIZE, is_train=True)
valid_generator = My_Generator(valid_x, valid_y, BATCH_SIZE, is_train=False)

In [ ]:
def create_model_xception(input_shape, n_out):
    input_tensor = Input(shape=input_shape)
    base_model = Xception(include_top=False,
                   weights=None,
                   input_tensor=input_tensor)
    base_model.load_weights('../input/keras-pretrained-models/xception_weights_tf_dim_ordering_tf_kernels_notop.h5')
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    final_output = Dense(n_out, activation='softmax', name='final_output')(x)
    model = Model(input_tensor, final_output)
    
    return model
model = create_model_xception((SIZE,SIZE,3), 5)
model.compile(Adam(1e-5), loss=kappa_loss,metrics=['accuracy' ])
model.summary()

In [ ]:
from keras.callbacks import Callback
class QWKEvaluation(Callback):
    def __init__(self, validation_data=(), batch_size=BATCH_SIZE, interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.batch_size = batch_size
        self.valid_generator, self.y_val = validation_data
        self.history = []

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict_generator(generator=self.valid_generator,
                                                  steps=np.ceil(float(len(self.y_val)) / float(self.batch_size)),
                                                  workers=1, use_multiprocessing=True,
                                                  verbose=1)
            def flatten(y):
                return np.argmax(y, axis=1).reshape(-1)
                # return np.sum(y.astype(int), axis=1) - 1
            
            score = cohen_kappa_score(flatten(self.y_val),
                                      flatten(y_pred),
                                      labels=[0,1,2,3,4],
                                      weights='quadratic')
#             print(flatten(self.y_val)[:5])
#             print(flatten(y_pred)[:5])
            print("\n epoch: %d - QWK_score: %.6f \n" % (epoch+1, score))
            self.history.append(score)
            if score >= max(self.history):
                print('save checkpoint: ', score)
                self.model.save('../working/Resnet50_bestqwk.h5')

qwk = QWKEvaluation(validation_data=(valid_generator, valid_y),
                    batch_size=BATCH_SIZE, interval=1)

In [ ]:
model.fit_generator(train_generator, steps_per_epoch=math.ceil(len(train_x)/BATCH_SIZE), 
                    epochs=15, validation_steps=math.ceil(len(valid_x)/BATCH_SIZE), callbacks=[qwk])